In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor, plot_tree

sys.path.append(os.path.abspath('..'))
from processamento import df, categorical_cols, numeric_cols

In [ ]:
# Considerar apenas Southern Europe
df_southern = df[df['Region'] == 'Southern Europe']

In [ ]:
# ---------------- Definir o KFold (para todos os modelos) ----------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Variàvel alvo global
y = df_southern['Premature_Deaths'].values

# **4.2.2 Modelos de regressão linear simples para prever mortes prematuras**

#### Treino do modelo linear com K-Fold Cross Validation e função linear

In [ ]:
# Variável não global de interesse para a regressão linear simples
X = df_southern[['Affected_Population']].values

model = LinearRegression()

mae_scores = []
rmse_scores = []
coefs = []
intercepts = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    
    coefs.append(model.coef_[0])
    intercepts.append(model.intercept_)

print("a) Função linear média resultante:")
print(f"y = {np.mean(coefs):.6f} * X + {np.mean(intercepts):.4f}")

#### Visualização: reta da regressão + diagrama de dispersão

In [ ]:
# Ajustar no conjunto total para visualização
model.fit(X, y)
y_pred_all = model.predict(X)

plt.figure(figsize=(10,6))
plt.scatter(X, y, color='blue', label='Dados reais', alpha=0.6)
plt.plot(X, y_pred_all, color='red', label='Regressão Linear')
plt.xlabel('Affected Population')
plt.ylabel('Premature Deaths')
plt.title('Regressão Linear Simples: Affected Population vs Premature Deaths (Southern Europe)')
plt.legend()
plt.grid(True)
plt.show()

#### Cálculo final de MAE e RMSE médios (validação cruzada)

In [ ]:
print("c) Erro Médio Absoluto (MAE): {:.2f}".format(np.mean(mae_scores)))
print("   Raiz do Erro Quadrático Médio (RMSE): {:.2f}".format(np.mean(rmse_scores)))

# **4.2.3 Desenvolver modelos para prever mortes prematuras**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),      # Normalização das variáveis numéricas
        ('cat', OneHotEncoder(), categorical_cols)    # Codificação one-hot das variáveis categóricas
    ]
)

## a) Regressão linear múltipla.

In [ ]:
# Variável não global de interesse para a regressão linear múltipla
X = df_southern.drop(columns=['Premature_Deaths'])

# Modelo de regressão linear múltipla
model_lr = LinearRegression()

# Pipeline: pré-processamento + modelo
pipeline_lrm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model_lr)
])

#### Treino do modelo de Regressão Linear Múltipla

In [ ]:
# Variáveis para armazenar as métricas
mae_lr, rmse_lr, r2_lr, residuals_lr = [], [], [], []

# Treinar e avaliar o modelo de Regressão Linear Múltipla com KFold
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    pipeline_lrm.fit(X_train, y_train)
    y_pred = pipeline_lrm.predict(X_test)
    mae_lr.append(mean_absolute_error(y_test, y_pred))
    rmse_lr.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    r2_lr.append(r2_score(y_test, y_pred))
    residuals_lr.append(y_test - y_pred)

# ---------------- Resultados globais ----------------
print(f"{'\nModelo':<30}{'MAE':>10}{'RMSE':>10}{'R2':>10}")
print(f"{'Regressão Linear Múltipla':<30}{np.mean(mae_lr):>10.2f}{np.mean(rmse_lr):>10.2f}{np.mean(r2_lr):>10.2f}")

# ---------------- Métricas por fold ----------------
print("\nMétricas por fold:")
for i, (mae, rmse, r2) in enumerate(zip(mae_lr, rmse_lr, r2_lr), 1):
    print(f"Fold {i}: MAE = {mae:.2f} | RMSE = {rmse:.2f} | R2 = {r2:.2f}")

#### Visualização: reta da regressão + distribuição dos erros

In [ ]:
# ---------------- Equações do modelo ----------------
# Ajustar o modelo aos dados completos para obter coeficientes finais
pipeline_lrm.fit(X, y)
coefs = pipeline_lrm.named_steps['model'].coef_
intercept = pipeline_lrm.named_steps['model'].intercept_
feature_names = (
    numeric_cols +
    list(pipeline_lrm.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))
)

# Equação APENAS com variáveis numéricas (mas o modelo inclui categóricas)
num_indices = [i for i, f in enumerate(feature_names) if f in numeric_cols]
equacao_numericas = "y = {:.2f} + ".format(intercept)
for i in num_indices:
    equacao_numericas += "{:.2f} * {} + ".format(coefs[i], feature_names[i])
equacao_numericas = equacao_numericas.rstrip(" +")
print("\nEquação (apenas variáveis numéricas):")
print(equacao_numericas)

In [ ]:
# ---------------- Visualização da Distribuição dos Erros ----------------
fig, axs = plt.subplots(1, 3, figsize=(15,4))
axs[0].boxplot(mae_lr)
axs[0].set_title('MAE ')
axs[1].boxplot(rmse_lr)
axs[1].set_title('RMSE')
axs[2].boxplot(r2_lr)
axs[2].set_title('R2')
plt.tight_layout()
plt.show()

In [ ]:
# ---------------- Coeficientes do modelo ----------------
coef_df = pd.DataFrame({
    'Variável': feature_names,
    'Coeficiente': coefs
})

# Separar coeficientes numéricos e categóricos
coef_num = coef_df[coef_df['Variável'].isin(numeric_cols)]
coef_cat = coef_df[~coef_df['Variável'].isin(numeric_cols)]

# Ordenar coeficientes categóricos por valor absoluto
coef_cat['AbsCoef'] = coef_cat['Coeficiente'].abs()
coef_cat = coef_cat.sort_values(by='AbsCoef', ascending=False)

print("\nCoeficientes das variáveis numéricas:\n")
print(coef_num[['Variável', 'Coeficiente']].to_string(index=False))

print("\nTop 10 coeficientes das variáveis categóricas:\n")
print(coef_cat[['Variável', 'Coeficiente']].head(10).to_string(index=False))

## b) Árvore de regressão com otimização dos parâmetros do modelo

In [ ]:
# ---------------- Árvore de Regressão (com otimização de parâmetros) ----------------
pipeline_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

param_grid = {
    'regressor__max_depth': [2, 3, 4, 5, 6, 8, 10, None],
    'regressor__min_samples_split': [2, 4, 6, 8, 10],
    'regressor__min_samples_leaf': [1, 2, 4, 6]
}

grid_search = GridSearchCV(
    pipeline_tree,
    param_grid,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_search.fit(X, y)

print("\nMelhores parâmetros da árvore de regressão:")
print(grid_search.best_params_)

#### Treino do modelo de Árvore de Regressão

In [ ]:
# Resultados da árvore de regressão
mae_tree, rmse_tree, r2_tree, residuals_tree = [], [], [], []
best_params = grid_search.best_params_

# Treino e avaliação da árvore de regressão com os melhores parâmetros
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    tree = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', DecisionTreeRegressor(
            random_state=42,
            max_depth=best_params['regressor__max_depth'],
            min_samples_split=best_params['regressor__min_samples_split'],
            min_samples_leaf=best_params['regressor__min_samples_leaf']
        ))
    ])
    tree.fit(X_train, y_train)
    y_pred = tree.predict(X_test)
    mae_tree.append(mean_absolute_error(y_test, y_pred))
    rmse_tree.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    r2_tree.append(r2_score(y_test, y_pred))
    residuals_tree.append(y_test - y_pred)

print(f"\n{'Árvore de Regressão':<30}{'MAE':>10}{'RMSE':>10}{'R2':>10}")
print(f"{'':<30}{np.mean(mae_tree):>10.2f}{np.mean(rmse_tree):>10.2f}{np.mean(r2_tree):>10.2f}")

print("\nMétricas por fold (Árvore de Regressão):")
for i, (mae, rmse, r2) in enumerate(zip(mae_tree, rmse_tree, r2_tree), 1):
    print(f"Fold {i}: MAE = {mae:.2f} | RMSE = {rmse:.2f} | R2 = {r2:.2f}")

#### Visualização: Árvore de Regressão

In [ ]:
# Ajustar a árvore final aos dados completos para visualização
final_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(
        random_state=42,
        max_depth=best_params['regressor__max_depth'],
        min_samples_split=best_params['regressor__min_samples_split'],
        min_samples_leaf=best_params['regressor__min_samples_leaf']
    ))
])
final_tree.fit(X, y)

plt.figure(figsize=(18, 8))
plot_tree(
    final_tree.named_steps['regressor'],
    feature_names=(
        numeric_cols +
        list(final_tree.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_cols))
    ),
    filled=True,
    rounded=True,
    max_depth=3,  # Mostra apenas os primeiros níveis para facilitar leitura
    fontsize=10
)
plt.title("Árvore de Regressão Otimizada (níveis iniciais)")
plt.show()

## c) SVM com otimização do kernel

## d) Rede neuronal com otimização da configuração da rede